# Testing and learning Probit/ Logit

Importing data and necessary library

In [76]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style='white')
sns.set(style="whitegrid", color_codes=True)

import pandas as pd
from sklearn.impute import KNNImputer

import pickle
from tqdm import tqdm

import zipfile
import io

#For reading Zip file
#Uncomment if you dont have th file yet
# zip_file_path = 'Data/College_Graduates.zip'
# sas_file_name = 'pcg21Public/epcg21.sas7bdat'

# # Open the ZIP file
# with zipfile.ZipFile(zip_file_path, 'r') as z:
#     # Extract the SAS file into memory
#     with z.open(sas_file_name) as sas_file:
#         # Read the SAS file from memory
#         df = pd.read_sas(io.BytesIO(sas_file.read()), format='sas7bdat')




USE PICKLE TO LOAD FILE WITHOUT RE-READING THEM. ALSO help with re-read variables 
Load data without reread the data. 

In [100]:
# ##pickle file to save:
# with open('college_data.pkl', 'wb') as file:
#     pickle.dump(df, file)
# Load the DataFrame from the pickle file
with open('college_data.pkl', 'rb') as file:
    df = pickle.load(file)

Check the data. College     # column's data type is bytes

In [101]:
for column in df.columns:
    # Check if the column's data type is bytes
    if df[column].dtype == 'O':  # 'O' stands for 'object', which can include bytes
        try:
            # Attempt to decode bytes to string
            df[column] = df[column].str.decode('utf-8')
        except AttributeError:
            print("fail\n")
            pass


Change logical skip and survey exclusion to nan, Gender column to fit, logical skip to nan

In [102]:

# Define the specific values to count

for column in df.columns:
    # Attempt to convert to numeric, ignore errors to keep other strings
    df[column] = pd.to_numeric(df[column], errors='ignore')

df = df.rename(columns={'GENDER': 'GENDER_male'})
df["GENDER_male"] = df["GENDER_male"].replace('M', "Y").replace('F', "N")

logic_skip_values = [np.nan, "X", "M", "L", "E", 98, 998, 9998, 999989, 999999,999999, 99998]
df = df.replace(logic_skip_values, np.nan).drop('COHORT', axis=1)



Preparing Data For analysis:
1. Change value to booleans
2. Change categorical values to dummy

Filter most missing value to eliminate var with amount missing larger than 25%. Let's check on labor force.

In [103]:
#Boolean
df = df.replace('L', np.nan).replace('N', 0).replace('Y', 1)
df['LFSTAT'] = df['LFSTAT'].replace({3: 0, 2: 0})


# Calculate the percentage of missing values for each column
missing_percentages = df.isnull().mean() * 100

# If data missing more than 25% then omit the variable
columns_to_keep = missing_percentages[missing_percentages <= 25].index
df_filtered = df[columns_to_keep]

numerical_cols = ['LFSTAT','AGE', 'AGEGR', 'BAACYR', 'BAAYR3', 'BAAYR5', 'BAYR', 'BIRYR', 'BSDGN', 'CH1218', 'CH19', 'CH25', 'CH6', 'CH611', 'CHU2', 'CLICYR', 'D23YR', 'D25YR', 'D2AYR', 'D2YR', 'D33YR', 'D35YR', 'D3AYR', 'D3YR', 'D43YR', 'D45YR', 'D4AYR', 'D4YR', 'D53YR', 'D55YR', 'D5AYR', 'D5YR', 'DGRYR', 'EARN', 'FNUSYR6', 'HDACY3', 'HDACYR', 'HDAY5', 'HSYR', 'LWYR', 'MR3YR', 'MR5YR', 'MRDACYR', 'MRYR', 'PERMVYR', 'SALARY', 'STRTYR']


THIS IS FOR IMPUTING DATA - FILLING MISSING DATA - take lots of time
See the data Disparity. Using combination of KNN and most frequent imputation method.

1. Most Frequent with large disparity of most frequent value (>70%)
2. KNN for rest of the value

In [85]:
from sklearn.impute import KNNImputer
import pandas as pd

# Calculate modes for all columns once
modes = df_filtered.mode().iloc[0]

# Impute most frequent for columns with a dominant category
dominant_columns = df_filtered.columns[df_filtered.apply(lambda col: (col.value_counts(normalize=True, dropna=False).iloc[0] > 0.7))]
df_filtered[dominant_columns] = df_filtered[dominant_columns].fillna(modes)

# Determine columns for KNN imputation
knn_columns = [col for col in df_filtered.columns if col not in numerical_cols and col not in dominant_columns]

if knn_columns:
    knn_imputer = KNNImputer()
    # Apply KNN imputation directly without looping through columns
    df_filtered[knn_columns] = knn_imputer.fit_transform(df_filtered[knn_columns])

# df_filtered now contains the imputed data


KNN Imputation:   0%|          | 0/106279 [07:14<?, ?it/s]
C:\Users\HoangLe.HoangLeLaptop\AppData\Local\Temp\ipykernel_24772\3631266363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[dominant_columns] = df_filtered[dominant_columns].fillna(modes)
C:\Users\HoangLe.HoangLeLaptop\AppData\Local\Temp\ipykernel_24772\3631266363.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[knn_columns] = knn_imputer.fit_transform(df_filtered[knn_columns])


Final Imputation (the remaining numerical variables)

In [ ]:
df_final_impute = df_final
df_final_impute[df_final_impute.columns] = knn_imputer.fit_transform(df_final_impute[df_final_impute.columns])
df_final_impute


,ACSIN,ACTCAP,ACTDED,ACTMGT,ACTRD,ACTRD2,ACTRDT,ACTRES,ACTTCH,AGE,...,BARGN_55,CTZUS_1,CTZUS_2,CTZUS_3,FACBEN_1,FACBEN_2,FACBEN_3,FACBEN_4,WKSYR_1,WKSYR_2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,40.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,34.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,32.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106274,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,26.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
106275,0.0,0.0,0.6,0.6,0.6,0.4,0.6,0.2,0.0,36.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
106276,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,35.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
106277,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,36.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


One hot encoded categorical data

In [93]:

#Categorical values to dummy
df_dummy = df_filtered

# Creating dummy variables for the categorical columns and drop orignal var
categorical_cols = df_dummy.select_dtypes(exclude=['number']).columns.difference(numerical_cols)
df_dummies = pd.get_dummies(df_dummy[categorical_cols])
df_final = pd.concat([df_dummy, df_dummies], axis=1)
df_final.drop(categorical_cols, axis=1, inplace=True)
df_final.head(5)

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

Checking Multicolinearity now.

Using Imperitive Impute from Scikit learn to impute. Maybe do KNN for faster time?
Take long time to impute

Can't use this method. Too long to impute. I have to consider missing data first

In [ ]:
# missing_percentages = df_final.isnull().mean() * 100
# # Convert to DataFrame for better display
# missing_df = pd.DataFrame(missing_percentages, columns=['% Missing Values'])

# import missingno as msno

# msno.matrix(df_final)

In [110]:
# #pickle file to save:
# with open('imputed_college_data.pkl', 'wb') as file:
#     pickle.dump(df_final_impute, file)
# import pickle
# # Load the DataFrame from the pickle file
with open('imputed_college_data.pkl', 'rb') as file:
    df_final_impute = pickle.load(file)

In [111]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

# Assuming 'df_final' is your DataFrame

# Set the independent variables, excluding 'LFSTAT'
X_columns = [col for col in df_final_impute.columns if col != 'LFSTAT']
X = df_final_impute[X_columns]

# VIF DataFrame
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

# Initialize list to store VIF for each feature
vif_values = []

# Wrap the loop with tqdm for a progress bar
for i in tqdm(range(len(X.columns)), desc='Calculating VIF'):
    # Calculate VIF for each feature in X
    vif = variance_inflation_factor(X.values, i)
    vif_values.append(vif)

# Assign the VIF values to the DataFrame
vif_data["VIF"] = vif_values

print(vif_data)
# vif_data.to_csv("vif_data.csv")

Calculating VIF:   6%|▋         | 16/251 [00:27<06:27,  1.65s/it]c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
Calculating VIF:  25%|██▍       | 62/251 [01:41<04:59,  1.58s/it]c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
Calculating VIF: 100%|██████████| 251/251 [06:43<00:00,  1.61s/it]

         feature           VIF
0          ACSIN  2.551171e+00
1         ACTCAP  5.568721e+00
2         ACTDED  8.672769e+00
3         ACTMGT  4.763844e+00
4          ACTRD  2.103495e+01
5         ACTRD2  1.303700e+01
6         ACTRDT  9.074725e+00
7         ACTRES  8.565181e+00
8         ACTTCH  5.639412e+00
9            AGE  2.821444e+03
10         AGEGR  9.964570e+01
11        ASDGRI  1.205237e+00
12         ASIAN  8.079298e+00
13        BAACYR  5.023458e+03
14        BAAYR3  3.870871e+02
15        BAAYR5  1.276948e+02
16       BADGRUS           inf
17         BAIND  1.166031e+00
18          BAMN  4.886989e+00
19          BAST  1.344012e+02
20          BAYR  5.561070e+03
21         BIRYR  2.778261e+03
22         BLACK  1.147640e+01
23         BSDGN  1.162761e+01
24        BTHRGN  2.656772e+01
25         BTHST  2.826011e+01
26         BTHUS  4.888993e+02
27        CHLVIN  1.341648e+00
28          CLIC  1.387552e+00
29       COMCOLI  1.282633e+00
30       CTZDUAL  1.446078e+00
31      

In [112]:
count = (vif_data["VIF"] > 10).sum()
count

113

Skip it. Do logit

Detect zero counts variables. Omit them

In [115]:
from tqdm import tqdm

# List to keep track of variables with zero-count cells
variables_to_drop = []
outcome_variable = 'LFSTAT'

df_final_impute['LFSTAT'] = df_final_impute['LFSTAT'].replace({3: 0, 2: 0})

for predictor in tqdm(df_final_impute.columns, desc='Analyzing Variables'):
    if predictor != outcome_variable:  # Skip the outcome variable itself
        # Create a crosstab
        ct = pd.crosstab(df_final_impute[outcome_variable], df_final_impute[predictor])
        
        # Check for any zero-count cells
        if (ct == 0).any().any():  # Checks if there are any zeros in the crosstab
            # print(f"Variable '{predictor}' has cells with zero counts.")
            variables_to_drop.append(predictor)

# Optionally, print or drop variables with zero-count cells
print("Variables with zero-count cells:", variables_to_drop)

df_reduced = df_final_impute.drop(columns=variables_to_drop)


Analyzing Variables:   3%|▎         | 7/252 [00:00<00:03, 65.62it/s]

Variable 'ACTCAP' has cells with zero counts.
Variable 'ACTDED' has cells with zero counts.
Variable 'ACTMGT' has cells with zero counts.
Variable 'ACTRD' has cells with zero counts.
Variable 'ACTRD2' has cells with zero counts.
Variable 'ACTRDT' has cells with zero counts.
Variable 'ACTRES' has cells with zero counts.
Variable 'ACTTCH' has cells with zero counts.


Analyzing Variables:   8%|▊         | 20/252 [00:00<00:04, 47.04it/s]

Variable 'BAACYR' has cells with zero counts.
Variable 'BAAYR3' has cells with zero counts.
Variable 'BAAYR5' has cells with zero counts.
Variable 'BAMN' has cells with zero counts.
Variable 'BAST' has cells with zero counts.


Analyzing Variables:  12%|█▏        | 30/252 [00:00<00:04, 44.81it/s]

...

Variable 'WKSWK' has cells with zero counts.
Variable 'WKTRNI' has cells with zero counts.
Variable 'WRKG' has cells with zero counts.


Analyzing Variables:  96%|█████████▋| 243/252 [00:49<00:04,  1.85it/s]

Variable 'WTSURVY' has cells with zero counts.


Analyzing Variables: 100%|██████████| 252/252 [00:49<00:00,  5.04it/s]

Variable 'WKSYR_1' has cells with zero counts.
Variable 'WKSYR_2' has cells with zero counts.
Variables with zero-count cells: ['ACTCAP', 'ACTDED', 'ACTMGT', 'ACTRD', 'ACTRD2', 'ACTRDT', 'ACTRES', 'ACTTCH', 'BAACYR', 'BAAYR3', 'BAAYR5', 'BAMN', 'BAST', 'BAYR', 'BTHRGN', 'BTHST', 'CLIC', 'DGRYR', 'EARN', 'EMED', 'EMRG', 'EMSECDT', 'EMSECSM', 'EMSIZE', 'EMSMI', 'EMST', 'EMTP', 'EMUS', 'ERNCOV1', 'FACADV', 'FACCHAL', 'FACIND', 'FACLOC', 'FACRESP', 'FACSAL', 'FACSEC', 'FACSOC', 'FTPRET', 'HDACYR', 'HDCARN', 'HDST', 'HRSWK', 'HSST', 'HSYR', 'JOBINS', 'JOBPENS', 'JOBPROFT', 'JOBSATIS', 'JOBVAC', 'JOBVACCOV', 'MGRNAT', 'MGROTH', 'MGRSOC', 'MRCARN', 'MRDACYR', 'MRST', 'MRYR', 'N2BAMED', 'N2BAMEDX', 'N2HDMEDX', 'N2MRMEDX', 'N2OCBLST', 'N2OCMLST', 'N2OCPRBG', 'N2OCPRMG', 'N3OCPR', 'N3OCPRNG', 'N3OCPRX', 'NBAMENG', 'ND5SEMG', 'NEDTP', 'NEWBUS', 'OBSNUM', 'OCEDRLP', 'PRMBR', 'RESPLO3', 'RESPLOC', 'SALARY', 'SALCOV1', 'SATADV', 'SATBEN', 'SATCHAL', 'SATIND', 'SATLOC', 'SATRESP', 'SATSAL', 'SATSEC',

Train Test Split

In [119]:
# df_final_temp['LFSTAT'] = df_final_temp['LFSTAT'].replace({3: 0, 2: 0})
X = df_reduced.drop('LFSTAT', axis=1)  # Features (independent variables)
y = df_reduced['LFSTAT']  # Target variable (dependent variable)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Predictive Logit and Probit

In [120]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the LogisticRegression model
logit_model = LogisticRegression(solver='liblinear')

# Fit the model to your data
logit_model.fit(X_train, y_train)

# Predictions
logit_predictions = logit_model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, logit_predictions))
print(classification_report(y_test, logit_predictions))


[[    0  3483]
 [    0 17773]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3483
         1.0       0.84      1.00      0.91     17773

    accuracy                           0.84     21256
   macro avg       0.42      0.50      0.46     21256
weighted avg       0.70      0.84      0.76     21256



c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Probit

In [121]:
# Initialize the LogisticRegression model with settings that approximate a Probit model
# Note: This remains an approximation; for exact Probit models, consider using statsmodels
probit_approx_model = LogisticRegression(solver='liblinear', penalty='l2')

# Fit the model to your data
probit_approx_model.fit(X_train, y_train)

# Predictions
probit_approx_predictions = probit_approx_model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, probit_approx_predictions))
print(classification_report(y_test, probit_approx_predictions))



[[    0  3483]
 [    0 17773]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3483
         1.0       0.84      1.00      0.91     17773

    accuracy                           0.84     21256
   macro avg       0.42      0.50      0.46     21256
weighted avg       0.70      0.84      0.76     21256



c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Descriptive Probit Logit

In [122]:
import statsmodels.api as sm

# Add a constant to the features (needed for statsmodels)
X_with_const = sm.add_constant(X)

# Create and fit the Probit model
probit_model = sm.Probit(y, X_with_const)
probit_results = probit_model.fit()

# Print the summary to see the coefficients
print(probit_results.summary())

         Current function value: 0.127895
         Iterations: 35


c:\Users\HoangLe.HoangLeLaptop\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          Probit Regression Results                           
Dep. Variable:                 LFSTAT   No. Observations:               106279
Model:                         Probit   Df Residuals:                   106157
Method:                           MLE   Df Model:                          121
Date:                Thu, 01 Feb 2024   Pseudo R-squ.:                  0.7120
Time:                        17:59:39   Log-Likelihood:                -13593.
converged:                      False   LL-Null:                       -47188.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ACSIN          -0.5517        nan        nan        nan         nan         nan
AGE            -0.0211        nan        nan        nan         nan         nan
AGEGR           0.0028        nan        nan    

In [ ]:
df_final.to_csv("df_final.csv")

In [123]:
# Create and fit the Logit model
logit_model = sm.Logit(y, X_with_const)
logit_results = logit_model.fit()

# Print the summary to see the coefficients
print(logit_results.summary())

Optimization terminated successfully.
         Current function value: 0.128164
         Iterations 16
                           Logit Regression Results                           
Dep. Variable:                 LFSTAT   No. Observations:               106279
Model:                          Logit   Df Residuals:                   106157
Method:                           MLE   Df Model:                          121
Date:                Thu, 01 Feb 2024   Pseudo R-squ.:                  0.7113
Time:                        17:59:52   Log-Likelihood:                -13621.
converged:                       True   LL-Null:                       -47188.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ACSIN          -1.1264    157.495     -0.007      0.994    -309.812     307.559
AGE            -0.0452   